<a href="https://colab.research.google.com/github/prachitshukla/DLFA-Kaggle/blob/development/M4_Mini_Hackathon_To_Perform_Classification_of_Coronavirus_Tweets_PS_GRU_0.77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint

### Mini Project Notebook: To perform text classification of coronavirus tweets during the peak Covid - 19 period using LSTMs/RNNs/CNNs/BERT.


## Learning Objectives

At the end of the mini-hackathon, you will be able to :

* perform data preprocessing/preprocess the text
* represent the text/words using the pretrained word embeddings - Word2Vec/Glove
* build the deep neural network (RNN, LSTM, GRU, CNNs, Bidirectional-LSTM, GRU, BERT) to classify the tweets


### Introduction

First we need to understand why sentiment analysis is needed for social media?

People from all around the world have been using social media more than ever. Sentiment analysis on social media data helps to understand the wider public opinion about certain topics such as movies, events, politics, sports, and more and gain valuable insights from this social data. Sentiment analysis has some powerful applications. Nowadays it is also used by some businesses to do market research and understand the customer’s experiences for their products or services.

Now an interesting question about this type of problem statement that may arise in your mind is that why sentiment analysis on COVID-19 Tweets? What is about the coronavirus tweets that would be positive? You may have heard sentiment analysis on movie or book reviews, but what is the purpose of exploring and analyzing this type of data?

The use of social media for communication during the time of crisis has increased remarkably over the recent years. As mentioned above, analyzing social media data is important as it helps understand public sentiment. During the coronavirus pandemic, many people took to social media to express their anger, grief, or sadness while some also spread happiness and positivity. People also used social media to ask their network for help related to vaccines or hospitals during this hard time. Many issues related to this pandemic can also be solved if experts considered this social data. That’s the reason why analyzing this type of data is important to understand the overall issues faced by people.



## Dataset

The given challenge is to build a multiclass classification model to predict the sentiment of Covid-19 tweets. The tweets have been pulled from Twitter and manual tagging has been done. We are given information like Location, Tweet At, Original Tweet, and Sentiment.

The training dataset consists of 36000 tweets and the testing dataset consists of 8955 tweets. There are 5 sentiments namely ‘Positive’, ‘Extremely Positive’, ‘Negative’, ‘Extremely Negative’, and ‘Neutral’ in the sentiment column.

## Description

This dataset has the following information about the user who tweeted:

1. **UserName:** twitter handler
2. **ScreenName:** a personal identifier on Twitter and is separate from the username
3. **Location:** where in the world the person tweets from
4. **TweetAt:** date of the tweet posted (DD-MM-YYYY)
5. **OriginalTweet:** the tweet itself
6. **Sentiment:** sentiment value



## Problem Statement

To build and implement a multiclass classification deep neural network model to classify between Positive/Extremely Positive/Negative/Extremely Negative/Neutral sentiments

## Grading = 10 Marks

Here is a handy link to Kaggle's competition documentation (https://www.kaggle.com/docs/competitions), which includes, among other things, instructions on submitting predictions (https://www.kaggle.com/docs/competitions#making-a-submission).

## Instructions for downloading train and test dataset from Kaggle API are as follows:

### 1. Create an API key in Kaggle.

To do this, go to the competition site on Kaggle at (https://www.kaggle.com/t/db0ea322e4b14ad1b29d14fbe406d4e5) and open your user settings page. Click Account.

* Click on your profile picture at the top-right corner of the page.

![alt text](https://i.imgur.com/kSLmEj2.png)

* In the popout menu, click the Settings option.

![alt text](https://i.imgur.com/tNi6yun.png)








### 2. Next, scroll down to the API access section and click generate to download an API key (kaggle.json).
![alt text](https://i.imgur.com/vRNBgrF.png)


### 3. Upload your kaggle.json file using the following snippet in a code cell:



In [ ]:
from google.colab import files
files.upload()

In [ ]:
#If successfully uploaded in the above step, the 'ls' command here should display the kaggle.json file.
%ls

### 4. Install the Kaggle API using the following command


In [ ]:
#!pip uninstall urllib3
#!pip install urllib3>=1.26.11
!pip install -U -q kaggle==1.5.8

#### 4.1 List of installed pakage

In [ ]:
!pip list

### 5. Move the kaggle.json file into ~/.kaggle, which is where the API client expects your token to be located:



In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
# Execute the following command to verify whether the kaggle.json is stored in the appropriate location: ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json # run this command to ensure your Kaggle API token is secure on colab

### 6. Now download the Test Data from Kaggle

**NOTE: If you get a '404 - Not Found' error after running the cell below, it is most likely that the user (whose kaggle.json is uploaded above) has not 'accepted' the rules of the competition and therefore has 'not joined' the competition.**

If you encounter **401-unauthorised** download latest **kaggle.json** by repeating steps 1 & 2

In [ ]:
#If you get a forbidden link, you have most likely not joined the competition.
!kaggle competitions download -c perform-classification-of-coronavirus-tweets

In [ ]:
!unzip /content/perform-classification-of-coronavirus-tweets.zip

## YOUR CODING STARTS FROM HERE

* install gensim

In [ ]:
# Run the below upgraded installations and restart the session
!pip install --upgrade numpy
!pip install --upgrade gensim

## Import required packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from gensim.utils import simple_preprocess

import tensorflow as tf  # use TensorFlow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, Embedding, Dense, Bidirectional, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import chardet

from matplotlib import pyplot as plt
import matplotlib
import matplotlib.patches as mpatches

import seaborn as sns
from wordcloud import WordCloud

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split


##   **Stage 1**:  Data Loading and Perform Exploratory Data Analysis (1 Points)

* Load the Dataset


In [ ]:
# Read the positive and negative files and split the sentences into a list
with open('corona_nlp_test.csv/corona_nlp_test.csv',"rb") as data_test:
  result = chardet.detect(data_test.read())
  print(result)
  data_test_set = pd.read_csv('corona_nlp_test.csv/corona_nlp_test.csv', encoding=result['encoding'])

with open('corona_nlp_train.csv/corona_nlp_train.csv',"rb") as data_train:
  result = chardet.detect(data_train.read())
  print(result)
  data_train_set = pd.read_csv('corona_nlp_train.csv/corona_nlp_train.csv', encoding=result['encoding'])

* check first 5 records of train dataframe

In [ ]:
print(data_train_set.head())

* Check for Missing Values

In [ ]:
print(data_train_set.isnull().sum())

* Visualize the sentiment column values


In [ ]:
print(data_train_set["Sentiment"])

In [ ]:
# Sentiment distribution bar plot
sentiment_count = data_train_set['Sentiment'].value_counts()
plt.figure(figsize=(10, 6))
plt.bar(sentiment_count.index, sentiment_count.values)
plt.title('Sentiments Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

* Visualize top 10 Countries that had the highest tweets using countplot (Tweet count vs Location)


In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data=data_train_set, x=data_train_set['Location'],  order= data_train_set['Location'].value_counts().iloc[:10].index)
plt.title('Top 10 Countries with the Highest Tweet Count')
plt.show()

* Plotting Pie Chart for the Sentiments in percentage


In [ ]:
plt.figure(figsize=(5,5))
sentiment_count={}
for sentiment in data_train_set['Sentiment'].unique():
  sentiment_count[sentiment]=data_train_set['Sentiment'].value_counts()[sentiment]
  #print(sentiment,data_train_set['Sentiment'].value_counts()[sentiment])
plt.pie(sentiment_count.values(), labels=sentiment_count.keys(), autopct='%1.1f%%')
plt.title('Distribution of Sentiments')
plt.axis('equal')
plt.show()

* WordCloud for the Tweets/Text

    * Visualize the most commonly used words in each sentiment using wordcloud
    * Refer to the following [link](https://medium.com/analytics-vidhya/word-cloud-a-text-visualization-tool-fb7348fbf502) for Word Cloud: A Text Visualization tool




In [ ]:
for sentiment in data_train_set['Sentiment'].unique():
  plt.figure(figsize=(20,5))
  text=' '.join(data_train_set['OriginalTweet'].astype(str))
  wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.title(sentiment, fontweight='bold')
  plt.axis("off")
  plt.show()
  print()


##   **Stage 2**: Data Pre-Processing  (2 Points)

####  Clean and Transform the data into a specified format


* convert categorical lables to numerical values

In [ ]:
le = LabelEncoder()
sentiments = ["Extremely Positive", "Positive",  "Neutral", "Negative", "Extremely Negative"]
le.fit(sentiments)
sentiment_to_num = {"Extremely Positive": 0,"Positive": 1,"Neutral": 2, "Negative": 3,"Extremely Negative": 4}
data_train_set['Sentiment_num'] = data_train_set['Sentiment'].map(sentiment_to_num)
data_train_set.head()

* function to preprocess the data

In [ ]:
data_train_set['OriginalTweet'] = data_train_set['OriginalTweet'].apply(lambda x:simple_preprocess(x, max_len=30))


In [ ]:
# Remove stop words
stop_words = set(stopwords.words('english'))

data_train_set['OriginalTweet'] = data_train_set['OriginalTweet'].apply(lambda x: [w for w in x if not w in stop_words])


In [ ]:
data_train_set.head()

### Hyper Parameters

In [ ]:
# Hyperparameters
MAX_SENT_LEN = 30   # Number of words to consider from each review
MAX_VOCAB_SIZE = 50000  # Max vocabulary size
BATCH_SIZE = 32
N_EPOCHS = 30

### Tokenize and Pad

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts([' '.join(seq[:MAX_SENT_LEN]) for seq in data_train_set['OriginalTweet']])

print("Number of words in vocabulary:", len(tokenizer.word_index))

In [ ]:
# Convert the sequence of words to sequnce of indices
X = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in data_train_set['OriginalTweet']])
X = pad_sequences(X, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

y = data_train_set['Sentiment_num']
#y_onehot = to_categorical(y, num_classes=5)
#y_onehot


### Spliting data to train and test set

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, stratify=y_onehot, random_state=42, train_size=30000)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, train_size=30000)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

##   **Stage 3**: Build the Word Embeddings using pretrained Word2vec/Glove (Text Representation) (1 Point)



* Import GloVe Embedding Files

In [ ]:
from IPython import get_ipython
ipython = get_ipython()
ipython.magic("sx wget -qq https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/glove.6B.zip")
ipython.magic("sx unzip glove.6B.zip")

In [ ]:
embeddings_index = {}
# Loading the 300-dimensional vector of the model
f = open('/content/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# Adding 1 because of reversed 0 index
words_not_found = []
vocab_size = len(tokenizer.word_index) + 1
print('Loaded %s word vectors.' % len(embeddings_index))

embedding_dim = 300

# Create a weight matrix for words in the training data
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
                embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)

In [ ]:
print(tokenizer.word_index)

In [ ]:
print(len(tokenizer.word_index))

##   **Stage 4**: Build and Train the Deep Recurrent Model using Pytorch/Keras (4 Points)



In [ ]:
# Build a sequential model by stacking neural net units
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights = [embedding_matrix],
                            input_length = MAX_SENT_LEN,
                            trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.5, name='first_gru_layer')))
model.add(Bidirectional(GRU(64, return_sequences=True, dropout=0.5, name='second_gru_layer')))
#model.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.5, name='second_gru_layer')))
#model.add(GRU(128, return_sequences=True, dropout=0.3, name='first_gru_layer'))
#model.add(Dropout(0.5))
model.add(Bidirectional(GRU(128, name='third_gru_layer', dropout=0.5)))
#model.add(Bidirectional(GRU(64, name='second_gru_layer', dropout=0.5)))
#model.add(GRU(64, name='second_gru_layer', dropout=0.3))
#model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(5, activation='sigmoid', name='output_layer'))
model.add(Dense(5, activation='softmax', name='output_layer'))

### Compile and Train Model

In [ ]:
#model.compile(loss='categorical_crossentropy',
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',           # or 'val_accuracy'
    patience=3,                   # stop after 3 epochs without improvement
    restore_best_weights=True,   # roll back to best weights
    verbose=1
)

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(X_test, y_test),
          callbacks=[early_stop])

In [ ]:
print('Summary of the built model...')
model.summary()

### Retrain Model on Full dataset

In [ ]:
model.fit(X, y)
print('Summary of the built model...')
model.summary()

##   **Stage 5**: Evaluate the Model and get model predictions on the test dataset (2 Points)


### Evaluate Model

In [ ]:
#Prepaire Testing Set
data_test_set['OriginalTweet'] = data_test_set['OriginalTweet'].apply(lambda x:simple_preprocess(x, max_len=30))

data_test_set['OriginalTweet'] = data_test_set['OriginalTweet'].apply(lambda x: [w for w in x if not w in stop_words])

#Convert Test Set
X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in data_test_set['OriginalTweet']])
X_test  = pad_sequences(X_test, maxlen=MAX_SENT_LEN, padding='post', truncating='post')




In [ ]:
# Get the prediction on the test data after training
test_predictions = model.predict(X_test)

In [ ]:
# Get the max values from prediction
predicted_test_classes = np.argmax(test_predictions, axis=1)
print(predicted_test_classes)
categorical_map={0:"Extremely Positive",1:"Positive",2:"Neutral",3:"Negative",4:"Extremely Negative"}
predicted_test_sentiments = [categorical_map[i] for i in predicted_test_classes]

# From the max values decode the sentiments
#predicted_test_sentiments = le.inverse_transform(predicted_test_classes)
#sentiment_to_num = {"Extremely Positive": 0,"Positive": 1,"Neutral": 2, "Negative": 3,"Extremely Negative": 4}

#print("Predicted sentiments (first 10):", predicted_test_sentiments[:10])

print(predicted_test_sentiments)

In [ ]:
# Get the test IDs
test_ids = data_test_set['UserName'].tolist()

# Create a dataframe and add the Test_Id and predicted sentiment text to it
output_df = pd.DataFrame({'Test_Id': test_ids, 'Sentiment': predicted_test_sentiments})

# Write to CSV file
output_df.to_csv('sentiment_predictions_AdditionalLayer_fulltrain_higher_dim.csv', index=False)

### Instructions for preparing Kaggle competition predictions


* Get the predictions using trained model and prepare a csv file
    * DeepNet model gives output for each class, consider the maximum value among all classes as prediction using `np.argmax`.

* Predictions (csv) file should contain 2 columns as Sample_Submission.csv
  - First column is the Test_Id which is considered as index
  - Second column is prediction in decoded form (for eg. Positive, Negative etc...).